# 8.1 Text generation with LSTM

## 8.1.3 The importance of the sampling strategy

In [ ]:
# Reweighting a probability distribution to a different temperature

import numpy as np

def reweight_distribution(original_distribution, temperature=0.5): # original_distribution is a 1D Numpy array of probability values that must sum to 1.
  distribution = np.log(original_distribution) / temperature
  distribution = np.exp(distribution)
  return distribution / np.sum(distribution) # Returns a reweighted version of the original distribution. The sum of the distribution may no longer be 1

## 8.1.4 Implementing character-level LSTM text generation

In [ ]:
# Downloading and parsing the initial text file

import keras
import numpy as np

path = keras.utils.get_file(
    "nietzsche.txt",
    origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
text = open(path).read().lower()
print("Corpus length:", len(text))

In [ ]:
# Vectorizing sequences of characters

maxlen = 60 # You'll extract sequences of 60 characters
step = 3 # You'll sample a new sequence every three characters
sentences = [] # Holds the extracted sequences
next_chars = [] # Holds the targets (the follow-up characters)

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i+maxlen])
  next_chars.append(text[i + maxlen])

print("Number of sequences:", len(sentences))

chars = sorted(list(set(text))) # List of unique characters in the corpus
print("Unique characters:", len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) # Dictionary that maps unique characters to their index in the list "chars"

print("Vectorization...")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

In [ ]:
# Single-layer LSTM model for next-character prediction

from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(llen(chars), activation="softmax"))

In [ ]:
# Model compilation configuration
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [ ]:
# Function to sample the next character given the model's predictions

def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype("float64")
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
# Text-generation loop

import random
import sys

for epoch in range(1, 60):
  print("epoch", epoch)
  model.fit(x, y, batch_size=128, epochs=1) # Fits the model for one iteration on the data
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  print('----Generating with seed: "' + generated_text + '"')

  for temperature in [0.2, 0.5, 1.0, 1.2]:
    print("---------temperature:", temperature)
    sys.stdout.write(generated_text)

  for i in range(400):
    samples = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
      sampled[0, t, char_index[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]

    generated_text += next_char
    generated_text = generated_text[1:]

    sys.stdout.write(next_char)

# 8.2 DeepDream

## 8.2.1 Implementing DeepDream in Keras

In [ ]:
# Loading the pretrained Inception V3 model

from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0) # You won't be training the model, so this command disables all training-specific operations

model = inception_v3.InceptionV3(weights="imagenet",
                                 include_top=False)

In [ ]:
# Setting up the DeepDream configuration

layer_contributions = {
    "mixed2": 0.2,
    "midex3": 3.,
    "mixed4": 2.,
    "mixed5": 1.5
}

In [ ]:
# Defining the loss to be maximized

layer_dict = dict([(layer.name layer) for layer in model.layers]) # Creates a dictionary that maps layer names to layer instances

loss = K.variable(0.) # You'll define the loss by adding layer contributions to this scalar variable
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output # Retrieves the layer's output

  scaling = K.prod(K.cast(K.shape(activation), "float32"))
  loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [ ]:
# Gradient-ascent process

dream = model.input # This tensor holds the generated image: the dream
grads = K.gradients(loss, dream)[0] # Computes the gradients of the dream with regard to the loss
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # Normalizes the gradients (important trick)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(X):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1]
  return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
  for i in range(iterations):
    loss_value, grad_values = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    print("...Loss value at", i, ":", loss_value)
    x += step * grad_values
  return x

In [ ]:
# Running gradient ascent over different successive scales

import numpy as np

step = 0.01 # Gradient ascent step size
num_octave = 3 # Number of scales at which to run gradient ascent
octave_scale = 1.4 # Size ratio between scales
iterations = 20 # Number of ascent steps to run at each scale

max_loss = 10. # If the loss grows larger than 10, you'll interrupt the gradient-ascent process to avoid ugly artifacts

base_image_path = "..."  # Fill this with the path to the image you want to use

img = preprocess_image(base_image_path) # Loads the base image into a Numpy array (function is defined in listing 8.13)
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
  shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
  successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1] # Reverses the list of shapes so they're in increasing order

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
  print("Processing image shape", shape)
  img = resize_img(img, shape) # Scales up the dream image
  img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)

  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
  same_size_original = resize_img(original_img, shape) # Computes the high-quality version of the original image at this size
  lost_detail = same_size_original - upscaled_shrunk_original_img # The difference between the two is the detail that was lost when scaling up

  img += lost_detail # Reinjects lost detail into the dream
  shrunk_original_img = resize_img(original_img, shape)
  save_img(img, fname="dream_at_scale_" + str(shape) + ".png")

save_img(img, fname="final_dream.png")

In [ ]:
# Auxiliary functions

import scipy
from keras.preprocessing import image

def resize_img(img, size):
  img = np.copy(img)
  factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)

  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  pil_img = deproces_image(np.copy(img))
  scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = inception_v3.preprocess_input(img)
  return img

def deprocess_img(x):
  if K.image_data_format() == "channels_first":
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1, 2, 0))

  else:
    x = x.reshape((x.shape[1], x.shape[2], 3))

  x /= 2.
  x += 0.5
  x *= 255.
  x = np.clip(x, 0, 255).astype("uint8")
  return x

# 8.3 Neural style transfer

## 8.3.3 Neural style transfer in Keras

In [ ]:
# Defining initial variables

from keras.preprocessing.image import load_img, img_to_array

target_image_path = "img/portrait.jpg" # Path to the image you want to transform
style_reference_image_path = "img/transfer_style_reference.jpg" # Path to the style image

width, height = load_img(targeT_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [ ]:
# Auxiliary functions

import numpy as np
from keras.apllications import vgg19

def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = vgg19.preprocess_input(img)
  return img

def deprocess_image(x):
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1] # Converts images from "BGR" to "RGB". This is also part of the reversal of vgg19.preprocess_input.
  x = np.clip(x, 0, 255).astype("uint8")
  return x

In [ ]:
# Loading the pretrained VGG19 network and applying it to the three images

from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3)) # Placeholder that will contain the generated image

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0) # Combines the three images in a single batch

model = vgg19.VGG19(input_tensor=input_tensor,
                    weights="imagenet",
                    include_top=False)
print("Model loaded.")

In [ ]:
# Content loss

def content_loss(base, combination):
  return K.sum(K.square(combination - base))

# Style loss

def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features))
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [ ]:
# Total variation loss

def total_variation_loss(x):
  a = K.square(
      x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :]
  )
  b = K.square(
      x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :]
  )
  return K.sum(K.pow(a + b, 1.25))

In [ ]:
# Defining the final loss that you'll minimize

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = "block5_conv2"
style_layers = ["block1_conv1",
                "block2_conv1",
                "block3_conv1",
                "block4_conv1",
                "block5_conv1"]
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss += (style_weight / len(style_layers)) * sl

loss += total_variation_weight * total_variation_loss(combination_image)

In [ ]:
# Setting up the gradient-descent process

grads = K.gradients(loss, combination_image)[0] # Gets the gradients of the generated image with regard to the loss

fetch_loss_and_grads = K.function([combination_image], [loss, grads]) # Function to fetch the values of the current loss and the current gradients

class Evaluator(object):

  def __init__(self):
    self.loss_value = None
    self.grads_values = None

  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads(x)
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype("float64")
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value

  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy()
    self.loss_value = None
    self.grad_values = None
    return grad_values

evaluator = Evaluator()

In [ ]:
# Style-transfer loop

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = "my_result"
iterations = 20

x = preprocess_image(target_image_path) # This is the initial state: the target image.
x = x.flatten()

for i in range(iterations):
  print("Start of iteration", i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                   x,
                                   fprime=evaluator.grads,
                                   maxfun=20)
  print("Current loss value:", min_val)
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_img(img)
  fname = result_prefix + "_at_iteration_%d.png" % i
  imsave(fname, img)
  print("Image saved as", fname)
  end_time = time.time()
  print("Iteration %d completed in %ds" % (i, end_time - start_time))

# 8.4 Generating images with variational autoencoders

## 8.4.3 Variational autoencoders

In [ ]:
# VAE encoder network

import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2 # Dimensionality of the latent space: a 2D plane

input_img = keras.Input(shape=img_shape)

x = layers.Conv2D(32, 3, padding="same", activation="relu")(input_img)
x = layers.Conv2D(64, 3, padding="same", activation="relu", strides=(2, 2))(x)
x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation="relu")(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [ ]:
# Latent-space-sampling function

def sampling(args):
  z_mean, z_log_var = args
  epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
  return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

In [ ]:
# VAE decoder network, mapping latent space points to images

decoder_input = layers.Input(K.int_shape(z)[1:]) # Input where you'll feed z

x = layers.Dense(np.prod(shape_before_flattening[1:]),
                 activtion="relu")(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3,
                           padding="same",
                           activation="relu",
                           strides=(2, 2))(x)
x = layers.Conv2D(1, 3, padding="same", activation="sigmoid")(x)
decoder = Model(decoder_input, x)
z_decoded = decoder(z)

In [ ]:
# Custom layer used to compute the VAE loss

class CustomVariationalLayer(keras.layers.Layer):

  def vae_loss(self, x, z_decoded):
    x = K.flatten(x)
    z_decoded = K.flatten(z_decoded)
    xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
    kl_loss = -5e-4 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean(xent_loss + kl_loss)

  def call(self, inputs):
    x = inputs[0]
    z_decoded = inputs[1]
    loss = self.vae_loss(x, z_decoded)
    self.add_loss(loss, inputs=inputs)
    return x # You don't use this output, but the layer must return something

In [ ]:
# Training the VAE

from keras.datasets import mnist

vae = Model(input_img, x)
vae.compile(optimizer="rmsprop", loss=None)
vae.summary()

(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype("float32") / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype("float32") / 255.
x_test = x_test.reshape(x_test.shape + (1,))

vae.fit(x=x_train, y=None,
        shuffle=True,
        epochs=10,
        batch_size=batch_size,
        validation_data=(x_test, None))

In [ ]:
# Sampling a grid of points from the 2D latent space and decoding them to images

import matplotlib.pyplot as plt
from scipy.stats import norm

n = 15
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
  for j, xi in enumerate(grid_y):
    z_sample = np.array([[xi, yi]])
    z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
    x_decoded = decoder.predict(z_sample, batch_size=batch_size)
    digit = x_decoded[0].reshape(digit_size, digit_size)
    figure[i * digit_size: (i+1) * digit_size,
           j * digit_size: (j+1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap="Greys_r")
plt.show()

# 8.5 Introduction to generative adversarial networks

## 8.5.3 The generator

In [ ]:
# GAN generator network

import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shaoe=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 5, strides=2, padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding="same")(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation="tanh", padding="same")(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

## 8.5.4 The discriminator

In [ ]:
# The GAN discriminator network

discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation="sigmoid")(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(
    lr = 0.0008,
    clipvalue = 1.0,
    decay = 1e-8
)

discriminator.compile(optimizer=discriminator_optimizer,
                      loss="binary_crossentropy")

## 8.5.5 The adversarial network

In [ ]:
# Adversarial network

discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr = 0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss="binary_crossentropy")

## 8.5.6 How to train your DCGAN

In [ ]:
# Implementing GAN training

import os
from keras.preprocessing import image

(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()
x_train = x_train[y_train.flatten() == 6] # Selects frog images (class 6)
x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype("float32") / 255.

iterations = 10000
batch_size = 20
save_dir = "your_dir"

start = 0
for step in range(iterations):
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim)) # Samples random points in the latent space
  generated_images = generator.predict(random_latent_vectors) # Decodes them to fake images

  stop = start + batch_size
  real_images = x_train[start: stop]
  combined_images = np.concatenate([generated_images, real_images]) # Combines them with real images
  labels = np.concatenate([np.ones((batch_size, 1)),
                           np.zeros((batch_size))])
  labels += 0.05 * np.random.random(labels.shape) # Adds random noise to the labels
  d_loss = discriminator.train_on_batch(combined_images, labels) # Trains the discriminator
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim)) # Samples random points in the latent space
  misleading_targets = np.zeros((batch_size, 1)) # Assembles labels that say "these are all real images"
  a_loss = gan.train_on_batch(random_latent_vectors, misleading_vectors) # Trains the generator (via the gan model, where the discriminator weights are frozen)

  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0

  if step % 100 == 0:
    gan.save_weights("gan.h5")

    print("discriminator loss:", d_loss)
    print("adversarial loss:", a_loss)

    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir, "generated_frog" + str(step) + ".png"))
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir, "real_fog" + str(step) + ".png"))